<a href="https://colab.research.google.com/github/computablee/DS-5899-CS-5260-Final-Project/blob/main/ResNet_vs_VIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

import numpy as np
import numpy.random as R
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from keras.utils.data_utils import get_file
from tensorflow.keras import models
from transformers import AutoImageProcessor, AutoModelForImageClassification, pipeline
from PIL import Image

(_, _), (test_images, test_labels) = mnist.load_data()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
test_images = test_images.astype('float32') / 255
nclassout = 10
test_labels_onehot = to_categorical(test_labels)

vis_test_images = np.expand_dims(test_images, axis=-1)
vis_test_images = np.repeat(vis_test_images, 3, axis=-1)
vis_test_images = tf.image.resize(vis_test_images, [224,224]).numpy()

In [3]:
model_path = get_file('resnet_mnist_digits.hdf5', 'https://huggingface.co/lane99/resnet_mnist_digits/resolve/main/resnet_mnist_digits.hdf5')
resnet = models.load_model(model_path)

evaluation_conv = resnet.evaluate(test_images, test_labels_onehot)

print("Residual Network Accuracy: ", str(evaluation_conv[1]))

313/313 [==============================] - 14s 35ms/step - loss: 0.0244 - accuracy: 0.9945
Residual Network Accuracy:  0.9944999814033508


In [4]:
device = torch.device('cuda')

image_processor = AutoImageProcessor.from_pretrained("farleyknight-org-username/vit-base-mnist")
model = AutoModelForImageClassification.from_pretrained("farleyknight-org-username/vit-base-mnist")

model.to(device)

success = 0
for i in range(len(vis_test_images)):
  inputs = image_processor(vis_test_images[i], return_tensors="pt")
  inputs.to(device)
  logits = model(**inputs).logits

  predicted_label = logits.cpu().data.numpy().argmax()
  if predicted_label == tf.math.argmax(test_labels_onehot[i]):
    success += 1

  if i > 0 and i % 1000 == 0:
    print("Accuracy: ", str(success / (i + 1)), ", Progress: ", str(i / len(vis_test_images)))

evaluation_vit = success / len(test_labels_onehot)

print("Vision Transformer Accuracy: ", str(evaluation_vit))

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Accuracy:  0.9820179820179821 , Progress:  0.1
Accuracy:  0.9850074962518741 , Progress:  0.2
Accuracy:  0.9863378873708764 , Progress:  0.3
Accuracy:  0.9872531867033242 , Progress:  0.4
Accuracy:  0.9882023595280944 , Progress:  0.5
Accuracy:  0.9896683886018997 , Progress:  0.6
Accuracy:  0.990001428367376 , Progress:  0.7
Accuracy:  0.9911261092363455 , Progress:  0.8
Accuracy:  0.9920008887901345 , Progress:  0.9
Vision Transformer Accuracy:  0.9922
